In [1]:
import pandas as pd 
from bs4 import BeautifulSoup 
from selenium.webdriver import Chrome
import pandas as pd
import re 
import time
import json
import math

C:\Users\teo-e\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
def get_page_number(keyword):
    #input: keyword for job_postings
    #output: number of pages

    url = base_url.format(keyword, 1)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    result_text = soup.find("span",{"class": "sx2jih0 zcydq84u _18qlyvc0 _18qlyvc1x _18qlyvc1 _1d0g9qk4 _18qlyvc8"})
    results = result_text.text.split()
    print(results)
    result = int(result_text.text.split()[-2].replace(',', ''))
    page_number = math.ceil(result/30)
    
    return page_number

In [3]:
def job_page_scraper(link):

    url = "https://www.jobstreet.com.sg"+link
    print("scraping...", url)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    scripts = soup.find_all("script")

    for script in scripts:
        if script.contents:
            txt = script.contents[0].strip()
            if 'window.REDUX_STATE = ' in txt:
                jsonStr = script.contents[0].strip()
                jsonStr = jsonStr.split('window.REDUX_STATE = ')[1].strip()
                jsonStr = jsonStr.split('}}}};')[0].strip()
                jsonStr = jsonStr+"}}}}"
                jsonObj = json.loads(jsonStr)
                
    job = jsonObj['details']
    job_id = job['id']
    job_expired = job['isExpired']
    job_confidential = job['isConfidential']
    job_salary_min = job['header']['salary']['min']
    job_salary_max = job['header']['salary']['max']
    job_salary_currency = job['header']['salary']['currency']
    job_title = job['header']['jobTitle']
    company = job['header']['company']['name']
    job_post_date = job['header']['postedDate']
    job_internship = job['header']['isInternship']
    company_website = job['companyDetail']['companyWebsite']
    company_avgProcessTime = job['companyDetail']['companySnapshot']['avgProcessTime']
    company_registrationNo = job['companyDetail']['companySnapshot']['registrationNo']
    company_workingHours = job['companyDetail']['companySnapshot']['workingHours']
    company_facebook = job['companyDetail']['companySnapshot']['facebook']
    company_size = job['companyDetail']['companySnapshot']['size']
    company_dressCode = job['companyDetail']['companySnapshot']['dressCode']
    company_nearbyLocations = job['companyDetail']['companySnapshot']['nearbyLocations']
    company_overview = job['companyDetail']['companyOverview']['html']
    job_description = job['jobDetail']['jobDescription']['html']
    job_summary = job['jobDetail']['summary']
    job_requirement_career_level = job['jobDetail']['jobRequirement']['careerLevel']
    job_requirement_yearsOfExperience = job['jobDetail']['jobRequirement']['yearsOfExperience']
    job_requirement_qualification = job['jobDetail']['jobRequirement']['qualification']
    job_requirement_fieldOfStudy = job['jobDetail']['jobRequirement']['fieldOfStudy']
    #job_requirement_industry = job['jobDetail']['jobRequirement']['industryValue']['label']
    job_requirement_skill = job['jobDetail']['jobRequirement']['skills']
    job_employment_type = job['jobDetail']['jobRequirement']['employmentType']
    job_languages = job['jobDetail']['jobRequirement']['languages']
    job_benefits = job['jobDetail']['jobRequirement']['benefits']
    job_apply_url = job['applyUrl']['url']
    job_location_zipcode = job['location'][0]['locationId']
    job_location = job['location'][0]['location']
    job_country = job['sourceCountry']

    return [job_id, job_title, job_expired, job_confidential, job_salary_min, job_salary_max, job_salary_currency, company, job_post_date, job_internship, company_website, company_avgProcessTime, company_registrationNo, company_workingHours, company_facebook, company_size, company_dressCode, company_nearbyLocations, company_overview, job_description, job_summary, job_requirement_career_level, job_requirement_fieldOfStudy, job_requirement_yearsOfExperience, job_requirement_qualification, job_requirement_skill, job_employment_type, job_languages, job_benefits, job_apply_url, job_location_zipcode, job_location, job_country]


In [4]:
def page_crawler(keyword, numOfPage=1):
    # input: keyword for job postings
    # output: dataframe of links scraped from each page

    # page number
#     page_number = get_page_number(keyword)
    job_links = []

#     for n in range(page_number):
    for n in range(numOfPage):
        print('Loading page {} ...'.format(n+1))
        url = base_url.format(keyword, n+1)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
        #extract all job links
        links = soup.find_all('a',{'class':'_1hr6tkx5'})
        job_links += links
 
    jobs = []

    for link in job_links:
        job_link = link['href'].strip().split('?', 1)[0]
        if "jobs" not in job_link:
            try:
                jobs.append([keyword, job_link] + job_page_scraper(job_link))
            except Exception as e:
                print("Error occured", e)

    result_df = pd.DataFrame(jobs, columns = ['keyword', 'link', 'job_id', 'job_title', 'job_expired', 'job_confidential', 'job_salary_min', 'job_salary_max', 'job_salary_currency', 'company', 'job_post_date', 'job_internship', 'company_website', 'company_avgProcessTime', 'company_registrationNo', 'company_workingHours', 'company_facebook', 'company_size', 'company_dressCode', 'company_nearbyLocations', 'company_overview', 'job_description', 'job_summary', 'job_requirement_career_level', 'job_requirement_fieldOfStudy', 'job_requirement_yearsOfExperience', 'job_requirement_qualification', 'job_requirement_skill', 'job_employment_type', 'job_languages', 'job_benefits', 'job_apply_url', 'job_location_zipcode', 'job_location', 'job_country'])
    return result_df

In [6]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'}
path = "C:/Users/teo-e/OneDrive/Desktop/SchoolWork/IS434 SA/project/chromedriver.exe"
driver = Chrome(executable_path=path)
time.sleep(2)
base_url = "https://www.jobstreet.com.sg/en/job-search/{}-jobs/{}/"

def main():

    # a list of job roles to be crawled
    key_words = ['software', 'data', 'computer']
#     dfs = []

    for key in key_words:
        key_df = page_crawler(key, numOfPage=30)
#         dfs.append(key_df)
        key_df.to_csv(key+"_job_postings_results.csv")
        print("----done scraping",key,"-----")
    
    # save scraped information as csv
#     pd.concat(dfs).to_csv("job_postings_results.csv")
    print("---scraping done---")

main()

Loading page 1 ...
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
scraping... https://www.jobstreet.com.sg/en/job/software-developer-9354937
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9407446
scraping... https://www.jobstreet.com.sg/en/job/software-qa-engineer-9409290
scraping... https://www.jobstreet.com.sg/en/job/senior-software-developer-9408408
scraping... https://www.jobstreet.com.sg/en/job/software-programmer-work-from-home-9310317
scraping..

scraping... https://www.jobstreet.com.sg/en/job/software-design-engineer-%5Bsemiconductor-c-north-area%5D-9408469
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9353449
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9399826
scraping... https://www.jobstreet.com.sg/en/job/associate-software-test-analyst-9360178
scraping... https://www.jobstreet.com.sg/en/job/software-tester-%5Bcentral-up-to-%2410000%5D-id%3A-543181-9380728
scraping... https://www.jobstreet.com.sg/en/job/games-software-engineer-unity-4870770/origin/my
scraping... https://www.jobstreet.com.sg/en/job/front-end-engineer-9357088
scraping... https://www.jobstreet.com.sg/en/job/junior-c-software-engineer-9406262
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-ref%3A-ans-4591-9344985
scraping... https://www.jobstreet.com.sg/en/job/.net-software-solution-consultant-.net-project-consultant-9409482
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9356498
scra

scraping... https://www.jobstreet.com.sg/en/job/software-developer-9363123
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-.net-oracle-pl-sql-powerbuilder-9405610
scraping... https://www.jobstreet.com.sg/en/job/r-d-embedded-software-engineer-190612br-%7C-worknow-9404637
scraping... https://www.jobstreet.com.sg/en/job/senior-software-developer-c-a-few-vacancies-9357002
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9398157
scraping... https://www.jobstreet.com.sg/en/job/software-operations-assistant-manager-9345409
scraping... https://www.jobstreet.com.sg/en/job/senior-software-engineer-cities-knowledge-graph-geo-applications-9397963
scraping... https://www.jobstreet.com.sg/en/job/software-test-analyst-tosca-9407570
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9368979
scraping... https://www.jobstreet.com.sg/en/job/software-specifications-engineer-9377335
scraping... https://www.jobstreet.com.sg/en/job/senior-business-analyst-sof

scraping... https://www.jobstreet.com.sg/en/job/junior-engineer-9315103
scraping... https://www.jobstreet.com.sg/en/job/devops-engineer-9400642
scraping... https://www.jobstreet.com.sg/en/job/software-developer-9395993
scraping... https://www.jobstreet.com.sg/en/job/software-quality-and-automation-engineer-%5Bwest-up-to-%246000%5D-id%3A-526004-9360825
scraping... https://www.jobstreet.com.sg/en/job/application-specialist-it-engineer-9358390
scraping... https://www.jobstreet.com.sg/en/job/marketing-manager-b2b-digital-it-software-9410795
scraping... https://www.jobstreet.com.sg/en/job/software-development-engineer-c-wpf-9398706
scraping... https://www.jobstreet.com.sg/en/job/software-manager-sap-fico-west-manufacturing-up-to-%249-000-9408444
scraping... https://www.jobstreet.com.sg/en/job/software-developer-52022-9397068
scraping... https://www.jobstreet.com.sg/en/job/software-test-engineer-api-testing-financial-industry-9406482
scraping... https://www.jobstreet.com.sg/en/job/lean-rpa-s

scraping... https://www.jobstreet.com.sg/en/job/software-tester-%5Beast-up-to-%244500%5D-id%3A-536350-9340686
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-9391309
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-product-development-18-months-contract-9365792
scraping... https://www.jobstreet.com.sg/en/job/product-developer-iot-9410129
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-iot-development-9293239
scraping... https://www.jobstreet.com.sg/en/job/flutter-developer-9407908
scraping... https://www.jobstreet.com.sg/en/job/it-project-manager-it-project-lead-software-manage-ict-9398784
scraping... https://www.jobstreet.com.sg/en/job/c-.net-application-engineer-9359503
scraping... https://www.jobstreet.com.sg/en/job/senior-lead-software-engineer-java-j2ee-9358477
scraping... https://www.jobstreet.com.sg/en/job/software-engineer%5B-java-struts-framework-ms-sql-tomcat-html-javascript%5D-9397419
scraping... https://www.jobstreet.com.

...

scraping... https://www.jobstreet.com.sg/en/job/it-support-engineer-%5Btroubleshoot-%7C-up-%243500-basic-%7C-east-%7C-5-days%5D-9396322
scraping... https://www.jobstreet.com.sg/en/job/software-engineer-%5Blabview-vision-python%5D-9306531
scraping... https://www.jobstreet.com.sg/en/job/enterprise-support-engineer-role-9404960
scraping... https://www.jobstreet.com.sg/en/job/it-support-executive-entertainment-media-mnc-9379602
scraping... https://www.jobstreet.com.sg/en/job/it-configuration-technician-9365537
scraping... https://www.jobstreet.com.sg/en/job/it-specialist-9349677
scraping... https://www.jobstreet.com.sg/en/job/it-support-engineer-1-year-contract-9364997
scraping... https://www.jobstreet.com.sg/en/job/it-officer-executive-f-b-ind-5days-9397478
scraping... https://www.jobstreet.com.sg/en/job/field-it-engineer-9393676
scraping... https://www.jobstreet.com.sg/en/job/information-system-is-manager-9335107
scraping... https://www.jobstreet.com.sg/en/job/it-desktop-support-3-months

scraping... https://www.jobstreet.com.sg/en/job/computer-vision-engineer-%EF%BC%88campus-recruitment-9355081
scraping... https://www.jobstreet.com.sg/en/job/it-analyst-9386096
scraping... https://www.jobstreet.com.sg/en/job/data-centre-operator-%7C-bishan-9393713
scraping... https://www.jobstreet.com.sg/en/job/it-support-engineer-9384174
scraping... https://www.jobstreet.com.sg/en/job/staff-engineer-mes-camstar-opcenter-architect-developer-9312181
scraping... https://www.jobstreet.com.sg/en/job/senior-software-engineer-scai-woopi-%3E-scai-1.0-9402760
scraping... https://www.jobstreet.com.sg/en/job/in-house-it-technician-9394710
scraping... https://www.jobstreet.com.sg/en/job/research-assistant-computer-science-electrical-electronic-engineering-environmental-earth-system-science-9359513
scraping... https://www.jobstreet.com.sg/en/job/it-team-lead-project-management-9308889
scraping... https://www.jobstreet.com.sg/en/job/it-executive-5-days-east-9391613
scraping... https://www.jobstreet.

scraping... https://www.jobstreet.com.sg/en/job/research-fellow-%7C-department-of-computer-science-9341529
scraping... https://www.jobstreet.com.sg/en/job/desktop-engineer-%7C-no-experience-needed-%7C-up-to-2k-9379931
scraping... https://www.jobstreet.com.sg/en/job/infrastructure-engineer-tech-stack-9339387
scraping... https://www.jobstreet.com.sg/en/job/senior-it-engineer-up-to-%248k-9395317
scraping... https://www.jobstreet.com.sg/en/job/mobile-phone-technicians-9316526
scraping... https://www.jobstreet.com.sg/en/job/scientific-it-officer-%7C-%5B3-year-renewable-contract%5D-urgent-9389628
scraping... https://www.jobstreet.com.sg/en/job/mis-executive-it-onsite-yearly-renewable-contract-5-days-%40-suntec-9379406
scraping... https://www.jobstreet.com.sg/en/job/it-desktop-onsite-support-system-network-engineers-9359286
scraping... https://www.jobstreet.com.sg/en/job/data-center-operations-technician-9313952
scraping... https://www.jobstreet.com.sg/en/job/senior-helpdesk-engineer-9387606


scraping... https://www.jobstreet.com.sg/en/job/project-executive-computer-electronic-mnc-east-9346642
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-computer-science-engineering-r00003938-9341638
scraping... https://www.jobstreet.com.sg/en/job/research-engineer-i-%5Bcomputer-engineering-electrical-electronic-engineering%5D.-9350341
scraping... https://www.jobstreet.com.sg/en/job/project-officer-computer-engineering-computer-science-electronics-engineering-9350034
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-%7C-%5Bcomputer-science-and-engineering%5D-9343931
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-computer-engineering-computer-science-9344852
scraping... https://www.jobstreet.com.sg/en/job/project-officer-computer-engineering-computer-science-r00003423-9337084
scraping... https://www.jobstreet.com.sg/en/job/senior-research-engineer-computer-science%5D-9350471
scraping... https://www.jobstreet.com.sg/en/job/it-helpdesk-9387211


scraping... https://www.jobstreet.com.sg/en/job/voice-engineer-9284703
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-computer-engineering-computer-science-electronics-engineering-9334240
scraping... https://www.jobstreet.com.sg/en/job/research-associate-computer-science-9340336
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-computer-science-engineering-r00006692-9333602
scraping... https://www.jobstreet.com.sg/en/job/research-assistant-computer-science-information-security-%7C-r00001878-9328102
scraping... https://www.jobstreet.com.sg/en/job/%5Bcomputer-science-electrical-electronic-engineering%5D-research-assistant-9333288
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-computer-science-%7C-r00002546-9339311
scraping... https://www.jobstreet.com.sg/en/job/project-officer-computer-science-electrical-electronics-engrng-r00006321-9333139
scraping... https://www.jobstreet.com.sg/en/job/research-fellow-computer-science-engineering-%7C-r000